In [2]:
%pip install torch torchvision torchaudio
%pip install -U pip setuptools wheel
!export CUDA_PATH="/opt/nvidia/cuda"
%pip install -U spacy[cuda11X,transformers]
%pip install transformers[sentencepiece]

%pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [15]:
import pandas as pd
sample_train = pd.read_csv(
    './train.csv', converters={'tag_set': eval, 'tag_list': eval})
sample_test = pd.read_csv(
    './test.csv', converters={'tag_set': eval, 'tag_list': eval})
sample_validation = pd.read_csv(
    './validation.csv', converters={'tag_set': eval, 'tag_list': eval})


In [16]:
import json
with open('selected_tags.json', 'r') as openfile:
    selected_tags = json.load(openfile)


In [17]:
from sklearn.model_selection import train_test_split
from tqdm import tqdm
import numpy as np
import spacy
from thinc.api import set_gpu_allocator, require_gpu

# Use the GPU, with memory allocations directed via PyTorch.
# This prevents out-of-memory errors that would otherwise occur from competing
# memory pools.
set_gpu_allocator("pytorch")
require_gpu(0)
nlp = spacy.load("en_core_web_trf")

def preprocess_transformers(texts):
    tokens = []
    removal = [ 'PUNCT', 'SPACE', 'NUM', 'SYM']
    cleaned_texts = []
    too_long = []
    print("preprocessing")
    for summary in tqdm(nlp.pipe(texts, disable=["transformer"]), total=len(texts)):
        question_tokens = []
        too_long.append(len(summary)>512)
        for token in summary:
            if token.pos_ not in removal and token.is_alpha and len(question_tokens)<512:
                question_tokens.append(token.lower_)
        cleaned_texts.append(" ".join(question_tokens))
    # question_tokens = [token.lemma_ for token in summary if token.pos_ not in removal and not token.is_stop and token.is_alpha]
    return cleaned_texts, too_long

def convert(data, outfile):
    db = spacy.tokens.DocBin()
    docs = []
    for doc, labels in tqdm(nlp.pipe(data.values, as_tuples=True), total=len(data)):
        for tag in selected_tags:
            doc.cats[tag] = tag in labels
        db.add(doc)

    db.to_disk(outfile)


In [6]:
train_df = sample_train[~sample_train.text.isna()].loc[:]

train_df["text_processed"], train_df["too_long"] = preprocess_transformers(train_df.text)
train_df = train_df[~train_df["too_long"]]
test_df = sample_test[~sample_test.text.isna()].sample(
    round(len(sample_test)/4))

test_df["text_processed"], test_df["too_long"] = preprocess_transformers(test_df.text)
test_df = test_df[~test_df["too_long"]]
validation_df = sample_validation[~sample_validation.text.isna()].loc[:]
validation_df["text_processed"], validation_df["too_long"] = preprocess_transformers(
    validation_df.text)
convert(train_df.loc[:, ["text_processed", "tag_list"]],
        'text_train_transformer.spacy')
convert(test_df.loc[:, ["text_processed",
        "tag_list"]], 'text_test_transformer.spacy')
convert(validation_df.loc[:, [
        "text_processed", "tag_list"]], 'text_validation_transformer.spacy')


preprocessing


100%|██████████| 5322/5322 [01:00<00:00, 87.44it/s] 


In [3]:
import torch
import gc
del nlp
gc.collect()
torch.cuda.empty_cache()


NameError: name 'nlp' is not defined

In [2]:
!python3 -m spacy train config_transformer_filled.cfg --output ./output_transformer --paths.train ./text_train_transformer.spacy --paths.dev ./text_test_transformer.spacy --gpu-id 0

2023-03-16 16:03:36.530246: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-16 16:03:38.978276: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-16 16:03:38.978632: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-16 16:03:38.978786: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [4]:
!python3 -m spacy evaluate ./output_transformer/model-best/ ./text_validation_transformer.spacy

2023-03-13 16:37:19.685659: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-13 16:37:21.254973: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 16:37:21.255323: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-13 16:37:21.255470: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [9]:
import spacy
texts = ["how to center a div", "list comprehension in python", "how to compare strings?"]
model = spacy.load("./output_transformer/model-best")
for text in texts:
    doc = model(preprocess_transformers([text])[0])
    print(doc.cats,  "-",  text)


preprocessing


100%|██████████| 1/1 [00:00<00:00, 6345.39it/s]


{'php': 0.006747276987880468, 'jquery': 0.37539854645729065, 'iphone': 0.04101081192493439, 'android': 0.010276096872985363, 'css': 0.9277873635292053, 'node.js': 0.006758995819836855, 'python': 0.004418166819959879, 'html': 0.8794304132461548, 'c': 0.013651935383677483, 'c++': 0.007743132766336203, 'objective-c': 0.02549903281033039, 'java': 0.007056664675474167, 'c#': 0.01749904453754425, 'ios': 0.03974779322743416, 'javascript': 0.2643504738807678, '.net': 0.010761790908873081} - how to center a div
preprocessing


100%|██████████| 1/1 [00:00<00:00, 6269.51it/s]


{'php': 0.0025349485222250223, 'jquery': 0.0012937092687934637, 'iphone': 0.006221460644155741, 'android': 0.0016969642601907253, 'css': 0.0027501462027430534, 'node.js': 0.000546167662832886, 'python': 0.9999946355819702, 'html': 0.0018443299923092127, 'c': 0.01143679954111576, 'c++': 0.00978111382573843, 'objective-c': 0.0014925593277439475, 'java': 0.0010312083177268505, 'c#': 0.0030155156273394823, 'ios': 0.0027126430068165064, 'javascript': 0.0020138206891715527, '.net': 0.004514699801802635} - list comprehension in python
preprocessing


100%|██████████| 1/1 [00:00<00:00, 2328.88it/s]

{'php': 0.06287722289562225, 'jquery': 0.036115482449531555, 'iphone': 0.05931819975376129, 'android': 0.030249858275055885, 'css': 0.012753098271787167, 'node.js': 0.04909913241863251, 'python': 0.10838162899017334, 'html': 0.029504921287298203, 'c': 0.03992507606744766, 'c++': 0.060569461435079575, 'objective-c': 0.10964897274971008, 'java': 0.10600638389587402, 'c#': 0.3700990676879883, 'ios': 0.08865784108638763, 'javascript': 0.08367820829153061, '.net': 0.3386959135532379} - how to compare strings?


In [18]:
train_df = sample_train[~sample_train.code.isna()].loc[:]
train_df["code_processed"], train_df["too_long"] = preprocess_transformers(train_df.code)
train_df = train_df[~train_df["too_long"]]
test_df = sample_test[~sample_test.code.isna()].sample(
    round(len(sample_test)/4))
test_df["code_processed"], test_df["too_long"] = preprocess(test_df.code)
test_df = test_df[~test_df["too_long"]]
validation_df = sample_validation[~sample_validation.code.isna()].loc[:]
validation_df["code_processed"], validation_df["too_long"] = preprocess(
    validation_df.code)
convert(train_df.loc[:, ["code_processed", "tag_list"]],
        'code_train_transformer.spacy')
convert(test_df.loc[:, ["code_processed",
        "tag_list"]], 'code_test_transformer.spacy')
convert(validation_df.loc[:, [
        "code_processed", "tag_list"]], 'code_validation_transformer.spacy')


preprocessing


100%|██████████| 76005/76005 [09:12<00:00, 137.64it/s]


In [7]:
import torch
import gc
del nlp
gc.collect()
torch.cuda.empty_cache()


: 

In [3]:
!python3 -m spacy train config_transformer_filled.cfg --output ./output_code_transformer --paths.train ./code_train_transformer.spacy --paths.dev ./code_test_transformer.spacy --gpu-id 0

2023-03-17 09:46:03.044001: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-17 09:46:04.556747: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 09:46:04.557120: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-17 09:46:04.557277: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [6]:
!python3 -m spacy evaluate ./output_code_transformer/model-best/ ./code_validation_transformer.spacy

2023-03-14 18:06:55.797552: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-14 18:06:57.359245: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 18:06:57.359791: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-14 18:06:57.360012: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least on

In [11]:
import spacy
texts = ["for i in range(3):", "foreach(p in toto)", "new String()"]
model = spacy.load("./output_code_transformer/model-best")
for text in texts:
    doc = model(preprocess_transformers([text])[0][0])
    print(doc.cats,  "-",  text)


preprocessing


100%|██████████| 1/1 [00:00<00:00, 209.09it/s]


{'php': 0.04501648619771004, 'jquery': 0.014227112755179405, 'iphone': 0.017142275348305702, 'android': 0.015957234427332878, 'css': 0.03041033260524273, 'node.js': 0.007229737471789122, 'python': 0.7023040652275085, 'html': 0.04644143208861351, 'c': 0.07310909032821655, 'c++': 0.07224539667367935, 'objective-c': 0.017172619700431824, 'java': 0.022521456703543663, 'c#': 0.017818061634898186, 'ios': 0.03495115041732788, 'javascript': 0.027802640572190285, '.net': 0.017179148271679878} - for i in range(3):
preprocessing


100%|██████████| 1/1 [00:00<00:00, 139.15it/s]


{'php': 0.051933884620666504, 'jquery': 0.02256924845278263, 'iphone': 0.02581527829170227, 'android': 0.026751745492219925, 'css': 0.05832083895802498, 'node.js': 0.00980730727314949, 'python': 0.31601426005363464, 'html': 0.07264366000890732, 'c': 0.044538650661706924, 'c++': 0.04212266951799393, 'objective-c': 0.024134976789355278, 'java': 0.025384197011590004, 'c#': 0.022086793556809425, 'ios': 0.05741535872220993, 'javascript': 0.04295099154114723, '.net': 0.023322148248553276} - foreach(p in toto)
preprocessing


100%|██████████| 1/1 [00:00<00:00, 153.44it/s]

{'php': 0.05628960207104683, 'jquery': 0.03808131068944931, 'iphone': 0.00596004631370306, 'android': 0.0940273255109787, 'css': 0.013288266025483608, 'node.js': 0.026030143722891808, 'python': 0.01955820433795452, 'html': 0.04948579892516136, 'c': 0.012252194806933403, 'c++': 0.02856658399105072, 'objective-c': 0.005427203141152859, 'java': 0.371188223361969, 'c#': 0.27540844678878784, 'ios': 0.00869848020374775, 'javascript': 0.09315021336078644, '.net': 0.160016268491745} - new String()


In [6]:
from tqdm import tqdm
import numpy as np
import spacy
from thinc.api import set_gpu_allocator, require_gpu
import pandas as pd
from spacy.scorer import Scorer
from spacy.training.example import Example
sample_validation = pd.read_csv(
    './validation.csv', converters={'tag_set': eval, 'tag_list': eval})
sample_validation.text.fillna("", inplace=True)
sample_validation.code.fillna("", inplace=True)
sample_validation["texts_processed"], _ = preprocess_transformers(sample_validation.text)
# sample_validation['codes_processed'], _ = preprocess(sample_validation.code)

set_gpu_allocator("pytorch")
require_gpu(0)
nlp_text = spacy.load("./output_transformer/model-best")
# scorer = Scorer(nlp_text)
# nlp_code = spacy.load("./output_code_transformer/model-best")
text_cats = []
code_cats = []
examples = []
for summary in tqdm(nlp_text.pipe(sample_validation["texts_processed"].values), total=len(sample_validation)):
    text_cats.append(summary.cats)

# for summary in tqdm(nlp_code.pipe(sample_validation["codes_processed"].values), total=len(sample_validation)):
#     code_cats.append(summary.cats)


preprocessing


100%|██████████| 14152/14152 [02:12<00:00, 106.90it/s]


In [7]:
# y = sample_validation.tag_list.apply(lambda tag_list: [1 if tag in tag_list else 0 for tag in selected_tags ])
y = sample_validation.tag_list.apply(
    lambda tag_list: [tag for tag in tag_list if tag in selected_tags])
y

0                            [html, css]
1                          [objective-c]
2                                  [ios]
3                                 [java]
4        [javascript, jquery, html, css]
                      ...               
14147                 [ios, objective-c]
14148                         [c#, .net]
14149              [iphone, objective-c]
14150                       [javascript]
14151                              [php]
Name: tag_list, Length: 14152, dtype: object

In [8]:
from sklearn.preprocessing import MultiLabelBinarizer
mlb = MultiLabelBinarizer()
y_t = mlb.fit_transform(y)
y_t


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [9]:
text_cats


[{'php': 0.0026427586562931538,
  'jquery': 0.1349508911371231,
  'iphone': 0.00414571026340127,
  'android': 0.019197018817067146,
  'css': 0.977730929851532,
  'node.js': 0.014476049691438675,
  'python': 0.004131522495299578,
  'html': 0.680175244808197,
  'c': 0.004158593248575926,
  'c++': 0.0035492596216499805,
  'objective-c': 0.0025896020233631134,
  'java': 0.009669819846749306,
  'c#': 0.0058102915063500404,
  'ios': 0.008402431383728981,
  'javascript': 0.23357966542243958,
  '.net': 0.005868055392056704},
 {'php': 0.004623806569725275,
  'jquery': 0.003919748589396477,
  'iphone': 0.5445550084114075,
  'android': 0.0023729174863547087,
  'css': 0.002053589327260852,
  'node.js': 0.002906654728576541,
  'python': 0.0028393215034157038,
  'html': 0.003930311184376478,
  'c': 0.008539254777133465,
  'c++': 0.006449208594858646,
  'objective-c': 0.5963723659515381,
  'java': 0.004132380243390799,
  'c#': 0.004829036071896553,
  'ios': 0.7934574484825134,
  'javascript': 0.00388

In [10]:
y_pred = [[x for x in selected_tags if y[x]>0.5] for y in text_cats]

y_pred


[['css', 'html'],
 ['iphone', 'objective-c', 'ios'],
 ['ios'],
 ['java', 'javascript'],
 ['jquery', 'css', 'html', 'javascript'],
 ['iphone', 'objective-c', 'ios'],
 ['iphone', 'ios'],
 ['ios'],
 ['android', 'java'],
 ['c#', '.net'],
 ['ios'],
 [],
 ['ios'],
 ['css', 'html'],
 ['html'],
 ['css'],
 ['c'],
 ['php'],
 ['java'],
 ['c#'],
 ['html'],
 ['html', 'javascript'],
 ['c#', '.net'],
 ['c#', '.net'],
 ['jquery', 'html', 'javascript'],
 [],
 ['python', 'javascript'],
 ['css', 'html', 'javascript'],
 ['c#'],
 ['python'],
 ['jquery', 'css', 'html'],
 ['android'],
 ['iphone', 'objective-c', 'ios'],
 ['javascript'],
 ['java'],
 [],
 ['c++'],
 ['python'],
 ['jquery', 'html', 'javascript'],
 ['jquery', 'html', 'javascript'],
 [],
 [],
 ['node.js'],
 ['c#', '.net'],
 ['android', 'java'],
 ['css', 'html', 'javascript'],
 ['php', 'jquery', 'javascript'],
 ['iphone'],
 [],
 ['c'],
 ['android', 'java'],
 ['html', 'c#'],
 ['c', 'c++'],
 [],
 ['jquery', 'css', 'html', 'javascript'],
 ['c#', '.net'

In [11]:
y_pred_t = mlb.transform(y_pred)
y_pred_t


array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 1, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 1, 0]])

In [12]:
from sklearn.metrics import roc_auc_score
roc_auc_score(y_t, y_pred_t, average=None)


array([0.76920958, 0.90808087, 0.84470962, 0.83880326, 0.78814831,
       0.91399607, 0.84160984, 0.87340877, 0.77884547, 0.79195218,
       0.84536337, 0.84987241, 0.87057117, 0.79323545, 0.86185151,
       0.90137628])

In [13]:

roc_auc_score(y_t, y_pred_t, average="macro")


0.8419396346698025

In [10]:
[np.array([y[x] for x in selected_tags]) for y in text_cats]


[array([0.00565867, 0.45037001, 0.00826579, 0.01454861, 0.96934301,
        0.00381946, 0.00994611, 0.63375837, 0.00937438, 0.02489178,
        0.01593969, 0.01261138, 0.03075662, 0.02672631, 0.4503265 ,
        0.0083988 ]),
 array([0.00791183, 0.01013499, 0.42437771, 0.0083115 , 0.00982516,
        0.0056808 , 0.00618995, 0.0099303 , 0.00935054, 0.00836404,
        0.66471612, 0.00927052, 0.00764628, 0.93413854, 0.01063169,
        0.00640766]),
 array([0.00546892, 0.00706505, 0.17901829, 0.00244036, 0.00839438,
        0.00384456, 0.00632441, 0.0069585 , 0.01061577, 0.01520297,
        0.54251218, 0.00476843, 0.00469877, 0.98249948, 0.0063741 ,
        0.00560273]),
 array([4.97087196e-04, 9.85621568e-03, 6.34017447e-03, 1.90290622e-02,
        9.22927484e-02, 5.94108924e-03, 4.46107658e-03, 5.28166816e-02,
        1.10373879e-03, 9.43368941e-04, 3.31468657e-02, 9.33930516e-01,
        6.34877128e-04, 4.32075150e-02, 2.02820793e-01, 1.18346920e-03]),
 array([0.01408573, 0.85172164, 

In [66]:
code_cats


[{'php': 0.49830642342567444,
  'jquery': 0.5004339814186096,
  'iphone': 0.5025267601013184,
  'android': 0.49954378604888916,
  'css': 0.4987199902534485,
  'node.js': 0.49792546033859253,
  'python': 0.4985634982585907,
  'html': 0.49974775314331055,
  'c': 0.498964786529541,
  'c++': 0.5000073909759521,
  'objective-c': 0.49860456585884094,
  'java': 0.49949637055397034,
  'c#': 0.49846264719963074,
  'ios': 0.4983639717102051,
  'javascript': 0.500343382358551,
  '.net': 0.4995996654033661},
 {'php': 0.49830642342567444,
  'jquery': 0.5004339814186096,
  'iphone': 0.5025267601013184,
  'android': 0.49954378604888916,
  'css': 0.4987199902534485,
  'node.js': 0.49792546033859253,
  'python': 0.4985634982585907,
  'html': 0.49974775314331055,
  'c': 0.498964786529541,
  'c++': 0.5000073909759521,
  'objective-c': 0.49860456585884094,
  'java': 0.49949637055397034,
  'c#': 0.49846264719963074,
  'ios': 0.4983639717102051,
  'javascript': 0.500343382358551,
  '.net': 0.499599665403366

In [20]:
[
    np.array([y[x] for x in selected_tags]) for y in code_cats]


[array([0.49830642, 0.50043398, 0.50252676, 0.49954379, 0.49871999,
        0.49792546, 0.4985635 , 0.49974775, 0.49896479, 0.50000739,
        0.49860457, 0.49949637, 0.49846265, 0.49836397, 0.50034338,
        0.49959967]),
 array([0.49830642, 0.50043398, 0.50252676, 0.49954379, 0.49871999,
        0.49792546, 0.4985635 , 0.49974775, 0.49896479, 0.50000739,
        0.49860457, 0.49949637, 0.49846265, 0.49836397, 0.50034338,
        0.49959967]),
 array([0.24377325, 0.08936024, 0.04217104, 0.03788395, 0.01344504,
        0.15237358, 0.05066327, 0.04752498, 0.04746091, 0.06934109,
        0.05654944, 0.03231623, 0.01689877, 0.11289   , 0.35016233,
        0.01620107]),
 array([0.49830642, 0.50043398, 0.50252676, 0.49954379, 0.49871999,
        0.49792546, 0.4985635 , 0.49974775, 0.49896479, 0.50000739,
        0.49860457, 0.49949637, 0.49846265, 0.49836397, 0.50034338,
        0.49959967]),
 array([0.01519131, 0.14723764, 0.01014235, 0.00526743, 0.97193795,
        0.00353067, 0.013676

In [21]:
sample_validation["text_cats"] = [
    np.array([y[x] for x in selected_tags]) for y in text_cats]
sample_validation["code_cats"] = [
    np.array([y[x] for x in selected_tags]) for y in code_cats]


In [32]:
sample_validation["cumulative_cats"] = sample_validation["text_cats"] + \
    sample_validation["code_cats"]


In [24]:
sample_validation["cumulative_cats"] = sample_validation.cumulative_cats.apply(lambda x: x/sum(x))

In [41]:
selected_tags

['php',
 'jquery',
 'iphone',
 'android',
 'css',
 'node.js',
 'python',
 'html',
 'c',
 'c++',
 'objective-c',
 'java',
 'c#',
 'ios',
 'javascript',
 '.net']

In [53]:
text_cats[4]

{'php': 0.014085734263062477,
 'jquery': 0.8517216444015503,
 'iphone': 0.014571968466043472,
 'android': 0.008243326097726822,
 'css': 0.7195079326629639,
 'node.js': 0.005831376649439335,
 'python': 0.008464419282972813,
 'html': 0.18920351564884186,
 'c': 0.007983144372701645,
 'c++': 0.012601782567799091,
 'objective-c': 0.04635462537407875,
 'java': 0.009478024207055569,
 'c#': 0.010503264144062996,
 'ios': 0.06380438804626465,
 'javascript': 0.4713280498981476,
 '.net': 0.006465306039899588}

In [54]:
code_cats[4]

{'php': 0.01519131101667881,
 'jquery': 0.1472376435995102,
 'iphone': 0.010142353363335133,
 'android': 0.005267426371574402,
 'css': 0.9719379544258118,
 'node.js': 0.0035306650679558516,
 'python': 0.013676239177584648,
 'html': 0.7666947245597839,
 'c': 0.012789676897227764,
 'c++': 0.007064380217343569,
 'objective-c': 0.007498963735997677,
 'java': 0.004064962733536959,
 'c#': 0.007132451981306076,
 'ios': 0.018808642402291298,
 'javascript': 0.1902339905500412,
 '.net': 0.007351102773100138}

In [55]:
sample_validation.iloc[4].tag_list


['javascript', 'jquery', 'html', 'css']

In [14]:
from collections import Counter
sample_validation.apply(lambda x: dict(Counter(x.text_cats)+Counter(y))) sample_validation["text_cats"]


0        {'php': 0.005658671259880066, 'jquery': 0.4503...
1        {'php': 0.007911826483905315, 'jquery': 0.0101...
2        {'php': 0.005468921270221472, 'jquery': 0.0070...
3        {'php': 0.0004970871959812939, 'jquery': 0.009...
4        {'php': 0.014085734263062477, 'jquery': 0.8517...
                               ...                        
14147    {'php': 0.005938441958278418, 'jquery': 0.0068...
14148    {'php': 0.0014416518388316035, 'jquery': 0.010...
14149    {'php': 0.5360350012779236, 'jquery': 0.042375...
14150    {'php': 0.008698887191712856, 'jquery': 0.9619...
14151    {'php': 0.9807122945785522, 'jquery': 0.038602...
Name: text_cats, Length: 14152, dtype: object